In [1]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
import re
import nltk
import nltk

data = pd.read_csv("https://raw.githubusercontent.com/amankharwal/Website-data/master/twitter.csv")
print(data.head())

   Unnamed: 0  count  hate_speech  offensive_language  neither  class  \
0           0      3            0                   0        3      2   
1           1      3            0                   3        0      1   
2           2      3            0                   3        0      1   
3           3      3            0                   2        1      1   
4           4      6            0                   6        0      1   

                                               tweet  
0  !!! RT @mayasolovely: As a woman you shouldn't...  
1  !!!!! RT @mleew17: boy dats cold...tyga dwn ba...  
2  !!!!!!! RT @UrKindOfBrand Dawg!!!! RT @80sbaby...  
3  !!!!!!!!! RT @C_G_Anderson: @viva_based she lo...  
4  !!!!!!!!!!!!! RT @ShenikaRoberts: The shit you...  


In [2]:
data.shape

(24783, 7)

In [3]:
nltk.download('stopwords')
stemmer = nltk.SnowballStemmer("english")
from nltk.corpus import stopwords
import string
stopword=set(stopwords.words('english'))

def clean(text):
    text = str(text).lower()
    text = re.sub('\[.*?\]', '', text)
    text = re.sub('https?://\S+|www\.\S+', '', text)
    text = re.sub('<.*?>+', '', text)
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub('\n', '', text)
    text = re.sub('\w*\d\w*', '', text)
    text = [word for word in text.split(' ') if word not in stopword]
    text=" ".join(text)
    text = [stemmer.stem(word) for word in text.split(' ')]
    text=" ".join(text)
    return text
data["tweet"] = data["tweet"].apply(clean)

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\HP\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [4]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer
nltk.download('vader_lexicon')
sentiments = SentimentIntensityAnalyzer()
data["Positive"] = [sentiments.polarity_scores(i)["pos"] for i in data["tweet"]]
data["Negative"] = [sentiments.polarity_scores(i)["neg"] for i in data["tweet"]]
data["Neutral"] = [sentiments.polarity_scores(i)["neu"] for i in data["tweet"]]
data['Compound'] = [sentiments.polarity_scores(i)["compound"] for i in data["tweet"]]
data.head()

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\HP\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


,Unnamed: 0,count,hate_speech,offensive_language,neither,class,tweet,Positive,Negative,Neutral,Compound
0,0,3,0,0,3,2,rt mayasolov woman shouldnt complain clean ho...,0.147,0.157,0.696,-0.0382
1,1,3,0,3,0,1,rt boy dat coldtyga dwn bad cuffin dat hoe ...,0.000,0.280,0.720,-0.5423
2,2,3,0,3,0,1,rt urkindofbrand dawg rt ever fuck bitch sta...,0.000,0.577,0.423,-0.8979
3,3,3,0,2,1,1,rt cganderson vivabas look like tranni,0.333,0.000,0.667,0.3612
4,4,6,0,6,0,1,rt shenikarobert shit hear might true might f...,0.154,0.407,0.440,-0.6808


In [6]:
data = data[["tweet", "Positive", 
             "Negative", "Neutral","Compound"]]
print(data.head())

                                               tweet  Positive  Negative  \
0   rt mayasolov woman shouldnt complain clean ho...     0.147     0.157   
1   rt  boy dat coldtyga dwn bad cuffin dat hoe  ...     0.000     0.280   
2   rt urkindofbrand dawg rt  ever fuck bitch sta...     0.000     0.577   
3             rt cganderson vivabas look like tranni     0.333     0.000   
4   rt shenikarobert shit hear might true might f...     0.154     0.407   

   Neutral  Compound  
0    0.696   -0.0382  
1    0.720   -0.5423  
2    0.423   -0.8979  
3    0.667    0.3612  
4    0.440   -0.6808  


In [15]:
Positive = data["Positive"].sum()
Negative = data["Negative"].sum()
Neutral = data["Neutral"].sum()
import plotly.express as px

labels = ['Positive','Negative','Neutral']
values = [Positive, Negative, Neutral]

fig = px.pie(data, values=values, names=labels, 
             title='Impressions on Instagram Posts From Various Sources', hole=0.5)
fig.show()

In [12]:
score = data["Compound"].values
sentimental = []
for i in score:
    if i >= 0.05 :
        sentimental.append('Positive')
    elif i <= -0.05 :
        sentimental.append('Negative')
    else:
        sentimental.append('Neutral')
data["Sentiment"] = sentimental
data.head()

,tweet,Positive,Negative,Neutral,Compound,Sentiment2,Sentiment
0,rt mayasolov woman shouldnt complain clean ho...,0.147,0.157,0.696,-0.0382,Neutral,Neutral
1,rt boy dat coldtyga dwn bad cuffin dat hoe ...,0.000,0.280,0.720,-0.5423,Negative,Negative
2,rt urkindofbrand dawg rt ever fuck bitch sta...,0.000,0.577,0.423,-0.8979,Negative,Negative
3,rt cganderson vivabas look like tranni,0.333,0.000,0.667,0.3612,Positive,Positive
4,rt shenikarobert shit hear might true might f...,0.154,0.407,0.440,-0.6808,Negative,Negative


In [14]:
data["Sentiment"].value_counts()

Negative    14663
Neutral      5067
Positive     5053
Name: Sentiment, dtype: int64

In [18]:
import plotly.express as px

# Assuming you have the "Sentiment" column in your DataFrame
sentiment_counts = data["Sentiment"].value_counts()
# Create a pie chart using Plotly
fig = px.pie(sentiment_counts, values=sentiment_counts.values, names=sentiment_counts.index, 
             title='Sentiment Distribution',hole=.5)
# Show the plot
fig.show()

In [7]:
x = sum(data["Positive"])
y = sum(data["Negative"])
z = sum(data["Neutral"])

In [19]:
def sentiment_score(a, b, c):
    if (a>b) and (a>c):
        print("Positive 😊 ")
    elif (b>a) and (b>c):
        print("Negative 😠 ")
    else:
        print("Neutral 🙂 ")
sentiment_score(x, y, z)

Neutral 🙂 
